In [1]:
%load_ext autoreload
%autoreload 2

import datetime
import numpy as np
import pandas as pd

from IPython.display import display

import CBOEFileParser
import VerticalSpreads
import CBOE_Downloader
import POP

import VerticalSpread

In [ ]:
underlying_symbol = "EWZ"
risk_free_rate = 2.94

## POP Calculator - only needs to be ran at the beginning of the trading day

In [ ]:
pop_calculator = POP.POP().get_ohlc_data(underlying_symbol)

# Update the Option Chain

In [2]:
# cboe_file = CBOE_Downloader.CBOE_Downloader(underlying_symbol).download().save_download().get_filename()
cboe_file = "/Users/glenn/Documents/CBOE_Data/BIDU_201807261013.dat"
# cboe_file = "/Users/glenn/Documents/CBOE_Data/GOOG_201807301217.dat"

option_chain_obj = CBOEFileParser.CBOEFileParser(
                    remove_options_after_days=60,
                    remove_options_before_days=30,
                    set_exchange=None) \
                    .parse_cboe_file(cboe_file=cboe_file) \
                    .add_greeks(risk_free_rate=0.298)

for expiration, calls, puts in option_chain_obj.items():
    print("Expiration: {}".format(expiration))
    print("Calls: {:,}".format(len(calls)))
    print("Puts: {:,}".format(len(puts))) 

Expiration: 2018-08-31 23:59:00
Calls: 47
Puts: 47
Expiration: 2018-09-07 23:59:00
Calls: 29
Puts: 29
Expiration: 2018-09-21 23:59:00
Calls: 35
Puts: 29


In [3]:
VerticalSpread.BullPutSpread(puts[20], puts[25]).valid()

True

In [4]:
VerticalSpread.BearCallSpread(calls[20], calls[25]).valid()

True

In [ ]:
pd.options.display.max_columns = None
vertical_spreads = VerticalSpreads.VerticalSpreads() \
                                    .build_spreads(option_chains)
#                                     .risk_limit(200) \
#                                     .itm_spreads(option_chains.underlying_spot) \
#                                     .add_pop(pop_calculator) \
#                                     .sky_view_spreads()
                
# vertical_spreads = VerticalSpreads.VerticalSpreads().build_spreads(option_chains).risk_limit(200).itm_spreads(option_chains.underlying_spot)
# vertical_spreads = VerticalSpreads.VerticalSpreads().build_spreads(option_chains)




print("Current Spot: {}".format(option_chains.underlying_spot))
for expiration, bull_calls, bull_puts, bear_calls, bear_puts in vertical_spreads.items(pretty_df=True):
    
    print("Expiration: {}".format(expiration))

    print("Bull Puts:")
    if isinstance(bull_puts, pd.DataFrame):
        display(bull_puts.sort_values(by=['Delta'], ascending=False).head(10))

    print("Bear Calls:")
    if isinstance(bear_calls, pd.DataFrame):
        display(bear_calls.sort_values(by=['Delta'], ascending=False).head(10))
